# Tutorial

This is a tutorial for using SPAMS - the Single Particle Model Solver.

For more advanced features please see the full documentation.

**Please note:** In this notebook, code to be run in the command line is preceded by an exclamation mark - this is not needed if running the program outside of this notebook.

In [1]:
# modules needed for tutorial
import user_input_mod as UI
import plotter
import sys

## Dependencies

*If you have read the dependencies section of the readme on github you can skip this section!*

In order to run the code (and this notebook), you will need to ensure you have the following installed:

- intel/2017.4.196-GCC-6.4.0-2.28
- impi/2017.3.196 imkl/2017.3.196
- imkl/2017.3.196
- netCDF-Fortran/4.4.4
- GCC/10.2.0
- Python/3.8.6
- numpy
- netCDF4
- matplotlib


For an scrtp managed system the following procedure will ensure correct dependendcies are installed:


    module purge; module load intel/2017.4.196-GCC-6.4.0-2.28  impi/2017.3.196 imkl/2017.3.196 netCDF-Fortran/4.4.4 GCC/10.2.0 Python/3.8.6
    pip3 install numpy netCDF4 Matplotlib


## Installing the code (downloading from git)

To install SPAMS, navigate to the directory in your file system where you would like to download it, and use the following command:

    git clone https://github.com/HetSys/PX915_GroupB_22-23

## Running the solver

The first time you use the program after downloading it from github, you will need to compile it. This can be done by navigating to the main program directory and using the make command.

In [2]:
! make

make: `finite_diff_solver' is up to date.


To run the program enter the following command:

In [3]:
! python3 user_input.py

set MKL_NUM_THREADS to 1
User input successful, calling solver...
 Success writing output file, user_input_output.nc                              
     

Solver executed successfully, plotting output...


## A Simple Input File

User input can be provided to the solver in two ways: by passing a simple input file containing a set of parameters or by passing a checkpoint file containing the state of a previous run at an intermediate timestep. The following are instructions to produce a simple input file for a half-cell model run in serial.

In order to use solver parameters specific to your system, you will need to edit the parameter values found in the user input file.

In the main directory locate the file called 'user_input.py' and open it with your preferred text editor.

Here, we will go through the main parts of this input file line by line.

### Input File Name

In order to produce a simple input file, 'checkpoint' should be set to 'False'.

The default name for the input file is 'user_input', this can be changed if desired. You should not add a file extension, and there is a characters limit of 50.

In [4]:
# Select input file: checkpoint file (True) or user input parameters (False).
# Enter the filename of desired checkpoint file or the desired name of the file containing user input parameters.
checkpoint = False
solver_input_filename = 'user_input'

### Input Parameters 

Below, is a list of all available input parameters. Default values are provided for parameters that do not change between the positive and negative electrodes.

- **tsteps**: The total number of timesteps for which the simulation is run. This must be an integer value greater than 0 (default = 100). 
*Do not change this if reading iapp from file.*

- **dt**: The size of the timestep size in seconds. This must be a float with a value greater than 0 (default = 0.1).

- **n**: The number of spatial nodes over which the simulation is run. This must be an integer value between 100 and 4000 (default = 1000).

- **c0**: Sets the initial concentration in mol m<sup>-3</sup>, This must be a float with a value greater than 0 (default = 1000.0).

- **D**: The diffusion constant in m<sup>2</sup> s<sup>-1</sup>. This must be a float.

- **R**: The radius of the sphere used in the SPM in m. This must be a float with a value greater than 0.

- **a**: Particle surface area per unit volume (m<sup>-1</sup>). This must be a float with a value greater than or equal to 0.

- **L**: Electrode thickness in meters. This must be a float with a value greater than or equal to 0.

- **iapp**: Applied current density in A m<sup>2</sup>. This must be a 1D array of floats of length tsteps. The default sets a constant current density of 0.73*10<sup>-3</sup>.

- **iapp_label**: Labels the type of applied current density, e.g. constant or from file string.

- **electrode_charge**: Label of electrode charge as positive or negative. (default = p for positive electrode, n for negative electrode).

### Setting parameters

For setting system parameters there are two main options:

1. You can use the default parameters for a positive or negative electrode by calling UI.set_defaults_pos() or UI.set_defaults_neg().

In [5]:
# Import default values
tsteps, dt, n, c0, D, R, a, L, iapp, iapp_label, electrode_charge = UI.set_defaults_pos()

The default values for parameters D, R, a, and L for positive and negative electrodes are taken from Chen et al. 2020, https://doi.org/10.1149/1945-7111/ab9050. 

2. You can manually set each of the parameters. 

    The block of code below can be used in place of importing the default values.

In [6]:
# Number of timesteps, integer, greater than 0. Do not change if read iapp from file.
tsteps = 100
# Timestep size (s), real, greater than 0
dt = 0.1
# Number of spatial nodes, integer, between 100 and 4000.
n = 1000
# Initial concentration (mol m**-3), real, positive
c0 = 1000.0
# Diffusion coefficient (m**2 s**-1), real
D = 4.0e-15
# Width of block (m), real, greater than 0
R = 5.86e-6
# Particle surface area per unit volume (m**-1), real, greater than 0
a = 3.40e5
# Electrode thickness (m), real, greater than 0
L = 75.6e-6

#### Applying current

For varying the applied current density (i<sub>app</sub>) there are the following additional options:

1. You can set a constant value for current density, 'iapp_const', using the code below.


In [7]:
# Constant current - set iapp_const to a constant float value.
iapp_const = 0.73*10**(-3)
iapp, iapp_label = UI.iapp_constant_setup(tsteps, iapp_const)

2. You can set a stepped current density, 'iapp_steps', from a 2D array of values and timesteps using the code below.

In [8]:
# Step function - create 2D array of step values and integer timesteps starting at timestep 0
iapp_steps = [[0.73*10**(-3), 0], [-0.73*10**(-3), int(tsteps/4)], [0.73*10**(-3), int(tsteps/2)]]
iapp, iapp_label = UI.iapp_step_setup(tsteps, iapp_steps)

3. The i<sub>app</sub> values can be read in from a csv file stored in the variable 'iapp_filename'.
(_This code block is provided as an example and will not run in this tutorial._)

In [9]:
# Read in applied current density from csv file
iapp_filename = 'example.csv'
iapp, iapp_label, tsteps = UI.iapp_read_csv(iapp_filename)

FileNotFoundError: [Errno 2] No such file or directory: 'example.csv'

4. A final option is to manually set the i<sub>app</sub> values,
(i<sub>app</sub> must be an array of floats with a length of tsteps.)

In [ ]:
###### Manually set up applied current ######
### Applied current (A m**2), real array of length tsteps

All input parameters should be set above the line:

In [ ]:
######### END SET VALUES #########

The code following this line in 'user_input.py' does not require editing by the user. This code validates the user input parameters, writes them to file, and calls the solver and plotter.

## Visualising Results

When the solver is run, it produces plots that are saved in the main directory. The plots produced are:

1. An animation showing the concentration of lithium as a function of time and space. This will be saved as 'concentration_animation.gif' and should look similar to the animation shown here.

![animation](example_plots/concentration_animation.gif)

2. A plot of the concentration profile at the end of the simulation, available as 'final_state.png'. An example is shown here.

![conc plot](example_plots/final_state.png)

3. Plots of both voltage and applied current over time. These are saved to the same file called 'Voltage Current Plot.png'.

![voltage current](example_plots/Voltage%20Current%20Plot.png)

## Checkpointing

When the solver is run, checkpoint files are produced containing all of the user input parameters and the state of the system at an intermediate timestep of the simulation.

The solver can be run by passing one of these checkpoint files. This will continue the run from the timestep at which the checkpoint was recorded.

To do so, 'checkpoint' should be set to 'True' and the name of the desired checkpoint file should be passed to 'solver_input_filename'. The code can then be run as usual.

(_This code block is provided as an example and will not run in this tutorial._)

In [ ]:
# Select input file: checkpoint file (True) or user input parameters (False).
# Enter the filename of desired checkpoint file or the desired name of the file containing user input parameters.
checkpoint = True
solver_input_filename = 'checkpoints_user_input/example_checkpoint.nc'

By default, checkpoint files are produced at intervals equivalent to 10% of the total timesteps.

The number of checkpoint files produced in a run can be changed by editing 'finite_diff_solver.f90'. Locate the call to the subroutine 'write checkpoint'. The subroutine will write a checkpoint file every 'freq' timesteps, which by default is 10% of tsteps. This can be changed by adding freq to the end of the call, as demonstrated below for 20 timesteps.

`CALL write_checkpoint(tstep, tsteps, dt, n, c, D, R, a_small, L, iapp, electrode_charge, cstorage, filename, 20)`

# Advanced Features

In addition to the basic half single particle model, the program has several additional features:

### Output Errors to file

The messages output to terminal by the solver can be output to a file called 'stdout.txt' by uncommenting the following line at the top of 'user_input.py':

In [10]:
# The stdout (command line output) can be output to a file. Uncomment the line below to use this option.
# sys.stdout = open('stdout.txt', 'w')

### Parallelism - Basic

Basic parallelism can be set up in the solver by passing an optional argument 'nprocs', as shown in the code below. By default, nprocs is set to 1. 
The number passed to 'nprocs' is the maximum number of processors that the user would like to use when running the code. The solver will decide how many threads to use automatically in order to maximise the runtime on the number of processors provided.

In [ ]:
# Call fortran solver
UI.call_solver(solver_input_filename, checkpoint, nprocs=4)

### Parallelism - Full Battery Simulation

When running a simulation of a full battery, both halves can be run in parallel as seperate non-communicating processes. This is done automatically for you by the solver, depending on how many processors you specify in the function call.

A example script to call a simple full cell simulation is provided in 'unit_test.py'.
Specifically, calling a full battery simulation uses the following line:

`UI.full_battery_simulation(output_filename_positive,output_filename_negative,nprocs=4)`

Note that:
- If nprocs = 1, the simulation will run in serial
- If nprocs = 2, the simulation will split the cell into two seperate parallel processes (one for anode and one for cathode)
- Any further increases on nprocs provided will lead to multi-threading in the matrix solve through the environment variable MKL_NUM_THREADS with the intel math kernel library. The program decides the optimum strategy of how to do this for you.

### Parallelism - Half cell GITT simulation

A GITT test (Galvanostatic Intermittent Titration Technique) is a type of test which provides a series of steps of current density to a cell, with long rests of no current density in between to allow for the concentration profile to even out. It is possible to calculate the concentration removed in each step prior to the simulation running and subsequently if the rest time is long enough, it is possible to know the initial concentration value for each seperate current step. This means that each current step can be run as a seperate instance of the solver on seperate processors, and then stitched back together after, allowing for perfect parallelism.

A script to run a Half cell GITT simulation is provided in 'user_input_parallel_simulation.py'. Note that you can provide any number of processors (nprocs), when calling the solver function line (below) and the code will decide automatically the correct number of threads to use to minimise code runtime, given the number of GITT current steps that are being applied.

`UI.GITT_half_cell(output_filename,nprocs,currents,start_times,run_times,wait_times,n,params)`



*It should be noted that before running the solver in parallel, the user should make sure they are familiar with using a simple input file for a half-cell model run in serial as the scripts are largely similar.*

### Parallelism - Full Battery GITT test

A script is also provided to run a GITT test in parallel with a full battery. This scripts outputs a gif animation of the applied current and voltage with time along side the concentrations in the anode and cathode. See 'user_input_full_battery_GITT.py'.

*It should be noted that before running the full battery solver, the user should make sure they are familiar with using a simple input file for a half-cell model run in serial as the scripts are largely similar.*

### Uncertainty Quantification

The program provides scripts for performing both sensitivity analysis and uncertainty propagation. These can be run using the commands:

    python3 sensitivity_analysis.py

    python3 Uncertainty_Propagation.py
    
**Note** This will require loading of additional modules. Alternatively, the UQ results can be found in the developer documentation.

### Appending a Run

The 'checkpointing' branch of the github repository contains additional features to continue a run from an output file using a new user input parameter file with additional applied current 'iapp' steps. The 'user_input.py' file contains comments to explain the use of this feature.

This branch can be access with the command:

`git checkout checkpointing`

The main code can be reaccessed with the command:

`git checkout main`


## Documentation for developers

Full documentation of the code is available through Doxygen at https://hetsys.github.io/PX915_GroupB_22-23/.